## BIOS470/570 Lecture 21 Introduction to mathematical models of biological systems

In [ ]:
from scipy import integrate
import numpy as np
import matplotlib.pyplot as plt

### Let's consider the simple model for protein production and degradation: $\frac{dy}{dt} = k - d*y$. We define the model using a function that can be called. These functions must always have the time as the first input (even if unused) and the variable for the integrator as the second. It returns the time derivative of the variable. 

In [ ]:
def simpleProteinModel(t,y):
    k = 1
    d = 1
    dy = k-d*y
    return dy

### Set up an integrator to do numerical integration of the model:

In [ ]:
r = integrate.ode(simpleProteinModel)

### Do the integration and plot the results. The integrator stores the current state of the system and you can move it forward to time T by doing r.integrate(T). We move the integrator forward in steps of dt = 0.1 and store the t values and integration results in numpy arrays: 

In [ ]:
t0 = 0
y0 = 0
r.set_initial_value(y0,t0)
t_end = 10
dt = 0.1
t_vals =np.array(t0)
y_vals = np.array(y0)
while r.successful() and r.t < t_end:
    t_vals = np.append(t_vals,r.t+dt)
    y_vals = np.append(y_vals,r.integrate(r.t+dt))

### Now plot the results

In [ ]:
plt.plot(t_vals,y_vals,'.-')
plt.xlabel('Time')
plt.ylabel('Concentration');

### Let's repeat with a different initial value y0 = 5

In [ ]:
t0 = 0
y0 = 5
r.set_initial_value(y0,t0)
t_end = 10
dt = 0.1
t_vals =np.array(t0)
y_vals = np.array(y0)
while r.successful() and r.t < t_end:
    t_vals = np.append(t_vals,r.t+dt)
    y_vals = np.append(y_vals,r.integrate(r.t+dt))

plt.plot(t_vals,y_vals,'.-')
plt.xlabel('Time')
plt.ylabel('Concentration');

### Note the value at the end is the same in either case. Whether the concentration rises or falls depends on where we start.

### What if we want the flexibility to change the parameters such as the production or degradation rate? Define the function for the integration with an addation input:

In [ ]:
def simpleProteinModelParams(t,y,args):
    k = args[0]
    d = args[1]
    dy = k-d*y
    return dy

### We can run this the same way with a call to r.set_f_params to set the additional parameters in the function:

In [ ]:
r = integrate.ode(simpleProteinModelParams)
t0 = 0
y0 = 0
r.set_f_params([2,0.5])
r.set_initial_value(y0,t0)
t_end = 10
dt = 0.1
t_vals =np.array(t0)
y_vals = np.array(y0)
while r.successful() and r.t < t_end:
    t_vals = np.append(t_vals,r.t+dt)
    y_vals = np.append(y_vals,r.integrate(r.t+dt))

plt.plot(t_vals,y_vals,'.-')
plt.xlabel('Time')
plt.ylabel('Concentration');

### Now let's consider the more complex example of an autoregulatory gene: $\frac{dx}{dt} = \frac{k_u + k_b x^2}{1+x^2}-x$. This has two parameters for the rates of transcription when the factor is not bound ($k_u$) and when it is bound ($k_b$)

In [ ]:
def autoRegulationModel(t,y,ku,kb):
    dy = (ku+kb*y**2)/(1+y**2)-y
    return dy

### Run with inital value 0.1

In [ ]:
r = integrate.ode(autoRegulationModel)
t0 = 0
y0 = 0.1
r.set_f_params(0,5)
r.set_initial_value(y0,t0)
t_end = 10
dt = 0.1
t_vals =np.array(t0)
y_vals = np.array(y0)
while r.successful() and r.t < t_end:
    t_vals = np.append(t_vals,r.t+dt)
    y_vals = np.append(y_vals,r.integrate(r.t+dt))

plt.plot(t_vals,y_vals,'.-')
plt.xlabel('Time')
plt.ylabel('Concentration');

### Run with inital value 1

In [ ]:
r = integrate.ode(autoRegulationModel)
t0 = 0
y0 = 1
r.set_f_params(0,5)
r.set_initial_value(y0,t0)
t_end = 10
dt = 0.1
t_vals2 =np.array(t0)
y_vals2 = np.array(y0)
while r.successful() and r.t < t_end:
    t_vals2 = np.append(t_vals2,r.t+dt)
    y_vals2 = np.append(y_vals2,r.integrate(r.t+dt))

plt.plot(t_vals2,y_vals2,'.-')
plt.xlabel('Time')
plt.ylabel('Concentration');

### Plot the time courses for the two different initial conditions on the same set of axes. 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(t_vals,y_vals,'.-')
ax.plot(t_vals2,y_vals2,'.-')
ax.set_xlabel('Time')
ax.set_ylabel('Concentration');

#### This is known as bistability - for the same system the final outcome can be different depending on where you are start. There are two different steady states and which one is reached depends on the past state of the system. 

### Let's see how this works by plotting the derivitive as a function of the concentration of the regulator. 

In [ ]:
xvals = np.arange(0,6,0.1)
zz = np.zeros(xvals.shape)
ku = 0
kb = 5
deriv = autoRegulationModel(0,xvals,ku,kb)
plt.plot(xvals,deriv,'.-')
plt.plot(xvals,zz,'-',color = 'k')
plt.xlabel('Concentration');
plt.xlabel('Derivative');


This crosses 0 three times, so there are 3 fixed points. 2 are stable and 1 is unstable. 

### Lower kb to 1, what does this look like now. 

In [ ]:
xvals = np.arange(0,6,0.1)
zz = np.zeros(xvals.shape)
ku = 0
kb = 1
deriv = autoRegulationModel(0,xvals,ku,kb)
plt.plot(xvals,deriv,'.-')
plt.plot(xvals,zz,'-',color = 'k')
plt.xlabel('Concentration');
plt.xlabel('Derivative');

Now there is only 1 fixed point at 0

### We can get a more holistic view by plotting the available steady states as a function of parameters. The steady states are defined by $\frac{dx}{dt} = \frac{k_u + k_b x^2}{1+x^2}-x$ which is the same as $x^3-k_bx^2+x-k_u = 0$. This can be solved with the np.roots function:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ku = 0
for kb in np.arange(0,4,0.05):
    rts = np.roots([1,-kb,1,-ku])
    rts = rts[rts == np.real(rts)]
    ax.plot(kb*np.ones(rts.shape),rts,'.',color = 'k')    
ax.set_xlabel('$k_b$')
ax.set_ylabel('Steady state concentration');

This is called a bifurcation diagram. It shows the steady states as a function of a parameter and importantly it shows where this changes. Here around $k_b=2$ there a bifurcation where two new fixed points appear. This is called a saddle node bifurcation.

### set $k_u = 0.1$ and remake the bifurcation diagram:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ku = 0.1
for kb in np.arange(0,4,0.05):
    rts = np.roots([1,-kb,1,-ku])
    rts = rts[rts == np.real(rts)]
    ax.plot(kb*np.ones(rts.shape),rts,'.',color = 'k')    
ax.set_xlabel('$k_b$')
ax.set_ylabel('Steady state concentration');

Now there are two bifurcations, one where two fixed points appear and the other where they disappear. 

Each bifurcation has a normal form, the simplest form of that bifurcation. For this saddle node it is $\frac{dx}{dt} = r+x^2$. This has two fixed points for $r<0$ and none for $r>0$